In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import numpy as np
from IPython.utils import io
import ta
import matplotlib.pyplot as plt
import matplotlib
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import multiprocessing as mp
import backtesting as btng
from strategy import ARSIstrat
from typing import List, Union 
from tqdm import tqdm
from PERMUTATIONS import get_permutation

In [ ]:
new_df = pd.read_csv('BATS_QQQ, 60_a45be.csv')
start = '2018-01-01'
end_excl = '2024-01-01'   # exclusive upper bound (covers all of 2018–2022)
new_df['time'] = pd.to_datetime(new_df['time'].astype(int), unit='s')
new_df.set_index('time', inplace=True)
new_df.sort_index(inplace=True)
new_df = new_df.loc[(new_df.index >= start) & (new_df.index < end_excl)].copy()
new_df = new_df.rename(columns={
    'open':'Open', 'high':'High', 'low':'Low', 'close':'Close', 'volume':'Volume'
})

bt = Backtest(new_df, ARSIstrat, cash=1_000_000, commission=0.0, spread=0.0001, finalize_trades=True)  
with io.capture_output():  
    stats = bt.run()
print(stats)
bt.plot() 

In [ ]:
stats, heatmap = bt.optimize(
        n_fast=range(5, 16),          # 5..15
        n_slow=range(10, 41, 2),      # 10..40 step 2
        n_vslow=range(40, 121, 5),    # 40..120 step 5 (the "very slow" you asked to include)
        maximize='Sharpe Ratio',
        return_heatmap=True,
        constraint=lambda p: p.n_fast < p.n_slow < p.n_vslow,
        #method='random', max_tries=1500, random_state=42,  # uncomment to speed up search
    )

Sharpe_Ratio, Return, Trades = stats[['Sharpe Ratio', 'Return [%]', '# Trades']].values
print(stats[['Sharpe Ratio', 'Return [%]', '# Trades']])


In [ ]:
best = stats._strategy
BNF = best.n_fast
BNS = best.n_slow
BNVS = best.n_vslow
params = {
    'n_fast':  int(BNF),
    'n_slow':  int(BNS),
    'n_vslow': int(BNVS),
    'sig_len': int(getattr(best, 'sig_len', 14)),
    'sl_pct':  float(getattr(best, 'sl_pct', 0.01)),
}

print(params)

In [ ]:
Sharpe_Ratio_OPTI_real, Return_OPTI_real, Trades = stats[['Sharpe Ratio', 'Return [%]', '# Trades']].values
print("In-sample Sarpe Ratio", Sharpe_Ratio_OPTI_real, "Best n_fast", BNF, "Best n_slow", BNS, "Best n_vslow", BNVS)


n_permutations = 1000
perm_better_count = 1
permuted_pfs = []
print("In-Sample MCPT")

for perm_i in tqdm(range(1, n_permutations)):

    train_perm = get_permutation(new_df)

    train_bt = Backtest(train_perm, ARSIstrat, cash=1_000_000, commission=0.0, spread=0.0001, finalize_trades=True) 

    stat_perm, _ = train_bt.optimize(
        n_fast=range(5, 16),          
        n_slow=range(10, 41, 2),      
        n_vslow=range(40, 121, 5),   
        maximize='Sharpe Ratio',
        return_heatmap=True,
        constraint=lambda p: p.n_fast < p.n_slow < p.n_vslow,
    )

    best_SR_perm, _, _ = stat_perm[['Sharpe Ratio', 'Return [%]', '# Trades']].values

    if best_SR_perm >= Sharpe_Ratio_OPTI_real:
        perm_better_count += 1

    permuted_pfs.append(best_SR_perm)

insample_mcpt_pval = perm_better_count / n_permutations
print(f"In-sample MCPT P-Value: {insample_mcpt_pval}")

plt.style.use('dark_background')
pd.Series(permuted_pfs).hist(color='blue', label='Permutations')
plt.axvline(Sharpe_Ratio_OPTI_real, color='red', label='Real')
plt.xlabel("Profit Factor")
plt.title(f"In-sample MCPT. P-Value: {insample_mcpt_pval}")
plt.grid(False)
plt.legend()
plt.show()